In [1]:
import pandas as pd
import json
import numpy as np
def load_jsonl(jsonl_fpath):
    with open(jsonl_fpath) as f:
        lines = f.read().splitlines()

    df = pd.DataFrame(lines)
    df.columns = ['json_element']
    df['json_element'].apply(json.loads)
    df = pd.json_normalize(df['json_element'].apply(json.loads))

    return df

df = load_jsonl('../data/mednli-1.0.0/mli_test_v1.jsonl')
df = df[['sentence1','sentence2','gold_label']]
df.head()

,sentence1,sentence2,gold_label
0,"In the ED, initial VS revealed T 98.9, HR 73, ...",The patient is hemodynamically stable,entailment
1,"In the ED, initial VS revealed T 98.9, HR 73, ...",The patient is hemodynamically unstable.,contradiction
2,"In the ED, initial VS revealed T 98.9, HR 73, ...",The patient is in pain.,neutral
3,63 year old woman with known history of hyperc...,the patient was in a MVC,entailment
4,63 year old woman with known history of hyperc...,the patient has no medical history,contradiction


In [98]:
# https://physionet.org/content/phenotype-annotations-mimic/1.20.03/

#drop cases where not sure.
mimic_iii_annot

,ROW_ID,ADVANCED.CANCER,ADVANCED.HEART.DISEASE,ADVANCED.LUNG.DISEASE,ALCOHOL.ABUSE,CHRONIC.NEUROLOGICAL.DYSTROPHIES,CHRONIC.PAIN.FIBROMYALGIA,DEMENTIA,DEPRESSION,DEVELOPMENTAL.DELAY.RETARDATION,NON.ADHERENCE,NONE,OBESITY,OTHER.SUBSTANCE.ABUSE,SCHIZOPHRENIA.AND.OTHER.PSYCHIATRIC.DISORDERS
0,17170,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,17170,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,50828,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,50828,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,52670,0,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2271,2787,0,1,1,0,0,0,0,0,0,0,0,0,0,0
2272,18731,0,0,0,1,0,0,0,0,0,1,0,0,0,0
2273,18731,0,0,0,1,0,0,0,0,0,1,0,0,0,0
2274,8180,0,1,0,0,0,0,0,1,0,0,0,0,0,0


In [57]:
notes_mimic_iii = pd.read_csv('../data/mimiciii-14/NOTEEVENTS.csv',usecols=['ROW_ID','TEXT'])
notes_mimic_iii #--> filter!

,ROW_ID,TEXT
0,174,Admission Date: [**2151-7-16**] Dischar...
1,175,Admission Date: [**2118-6-2**] Discharg...
2,176,Admission Date: [**2119-5-4**] D...
3,177,Admission Date: [**2124-7-21**] ...
4,178,Admission Date: [**2162-3-3**] D...
...,...,...
2083175,2070657,NPN\n\n\n#1 Infant remains in RA with O2 sats...
2083176,2070658,"Neonatology\nDOL #5, CGA 36 weeks.\n\nCVR: Con..."
2083177,2070659,Family Meeting Note\nFamily meeting held with ...
2083178,2070660,NPN 1800\n\n\n#1 Resp: [**Known lastname 2243*...


In [101]:
#excluding rows that are in the phenotype annotation task dataset
notes_mimic_iii_for_pretraining = notes_mimic_iii[~notes_mimic_iii['ROW_ID'].isin(mimic_iii_annot['ROW_ID'])]
notes_mimic_iii_for_pretraining

,ROW_ID,TEXT
0,174,Admission Date: [**2151-7-16**] Dischar...
1,175,Admission Date: [**2118-6-2**] Discharg...
2,176,Admission Date: [**2119-5-4**] D...
3,177,Admission Date: [**2124-7-21**] ...
4,178,Admission Date: [**2162-3-3**] D...
...,...,...
2083175,2070657,NPN\n\n\n#1 Infant remains in RA with O2 sats...
2083176,2070658,"Neonatology\nDOL #5, CGA 36 weeks.\n\nCVR: Con..."
2083177,2070659,Family Meeting Note\nFamily meeting held with ...
2083178,2070660,NPN 1800\n\n\n#1 Resp: [**Known lastname 2243*...


In [99]:
notes_mimic_iii_for_annot = notes_mimic_iii[notes_mimic_iii['ROW_ID'].isin(mimic_iii_annot['ROW_ID'])]
notes_mimic_iii_for_annot

,ROW_ID,TEXT
59,206,Admission Date: [**2189-2-18**] Dischar...
61,208,Admission Date: [**2189-3-17**] Dischar...
62,209,Admission Date: [**2189-3-17**] Dischar...
155,80,Admission Date: [**2195-2-5**] Discharg...
159,84,Admission Date: [**2195-6-29**] [**Year...
...,...,...
55607,53514,Admission Date: [**2123-10-12**] Discha...
55782,53748,Admission Date: [**2133-4-26**] ...
55783,53749,Admission Date: [**2134-1-16**] ...
55874,53339,Admission Date: [**2144-2-29**] ...


In [100]:
notes_mimic_iii_for_annot.merge(mimic_iii_annot, left_on='ROW_ID', right_on='ROW_ID',how='left')
#note that some of the instances are repeated with different 

,ROW_ID,TEXT,ADVANCED.CANCER,ADVANCED.HEART.DISEASE,ADVANCED.LUNG.DISEASE,ALCOHOL.ABUSE,CHRONIC.NEUROLOGICAL.DYSTROPHIES,CHRONIC.PAIN.FIBROMYALGIA,DEMENTIA,DEPRESSION,DEVELOPMENTAL.DELAY.RETARDATION,NON.ADHERENCE,NONE,OBESITY,OTHER.SUBSTANCE.ABUSE,SCHIZOPHRENIA.AND.OTHER.PSYCHIATRIC.DISORDERS
0,206,Admission Date: [**2189-2-18**] Dischar...,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,208,Admission Date: [**2189-3-17**] Dischar...,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,208,Admission Date: [**2189-3-17**] Dischar...,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,208,Admission Date: [**2189-3-17**] Dischar...,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,208,Admission Date: [**2189-3-17**] Dischar...,0,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1983,53749,Admission Date: [**2134-1-16**] ...,0,0,0,1,0,0,0,0,0,0,0,0,1,0
1984,53339,Admission Date: [**2144-2-29**] ...,1,0,0,0,0,1,1,0,0,0,0,0,0,0
1985,53339,Admission Date: [**2144-2-29**] ...,1,0,0,0,0,1,1,0,0,0,0,0,0,1
1986,53590,Admission Date: [**2115-3-6**] D...,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [ ]:
#####

In [25]:
classes = list(np.unique(df['gold_label']))
classes

['contradiction', 'entailment', 'neutral']

In [24]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", return_dict=True)

'''
The goal of the task is to classify a given premise-hypothesis pair into one of the three classes: entailment, contradiction, or neutral.
'''

classes = list(np.unique(df['gold_label']))

for i in range(len(df)):
    premise = df.iloc[i]['sentence1']
    hypothesis = df.iloc[i]['sentence2']
    true_label = df.iloc[i]['gold_label']
    
    tokenized_sequences = tokenizer(premise, hypothesis, return_tensors="pt")

    logits = model(**tokenized_sequences).logits
    result = torch.softmax(logits, dim=1).tolist()[0]

    for c in range(len(classes)):
        print(f"{classes[c]}: {int(round(result[c] * 100))}%")
        # if true_label == 'entailment':
        # elif label == 'contradiction':
        # elif label == 'neutral':

    # Should not be contradiction
    # for c in range(len(classes)):
    #     print(f"{classes[c]}: {int(round(not_paraphrase_results[c] * 100))}%")
    # # Should be neutral
    # for c in range(len(classes)):
    #     print(f"{classes[c]}: {int(round(not_paraphrase_results[c] * 100))}%")

    
# paraphrase_classification_logits = model(**paraphrase).logits
# not_paraphrase_classification_logits = model(**not_paraphrase).logits

# paraphrase_results = torch.softmax(paraphrase_classification_logits, dim=1).tolist()[0]
# not_paraphrase_results = torch.softmax(not_paraphrase_classification_logits, dim=1).tolist()[0]


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

IndexError: list index out of range

In [1]:
from torchtext import data
from torchtext import datasets
from transformers import AutoTokenizer

path = '../data/'

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

TEXT = data.Field(use_vocab=False, tokenize=tokenizer.encode)
LABEL = data.LabelField()

fields = {'sentence1': ('premise', TEXT),
          'sentence2': ('hypothesis', TEXT),
          'gold_label': ('label', LABEL)}

train, valid, test = data.TabularDataset.splits(
    path=path, 
    train='mli_train_v1.jsonl',
    validation='mli_dev_v1.jsonl',
    test='mli_test_v1.jsonl',
    format='json', 
    fields=fields
)

train_iter, valid_iter, test_iter = data.BucketIterator.splits(
    (train, valid, test), batch_sizes=(16, 256, 256)
)

pad_index = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
TEXT = data.Field(use_vocab=False, tokenize=tokenizer.encode, pad_token=pad_index) #making sure tokenizer vocab ONLY is used.
LABEL.build_vocab(train)

OSError: dlopen(/Users/simon/opt/anaconda3/lib/python3.7/site-packages/torchtext/_torchtext.so, 6): Symbol not found: __ZN3c104impl23ExcludeDispatchKeyGuardC1ENS_14DispatchKeySetE
  Referenced from: /Users/simon/opt/anaconda3/lib/python3.7/site-packages/torchtext/_torchtext.so
  Expected in: /Users/simon/opt/anaconda3/lib/python3.7/site-packages/torch/lib/libc10.dylib
 in /Users/simon/opt/anaconda3/lib/python3.7/site-packages/torchtext/_torchtext.so